# Cross Validation

Given that we trained and evaluated the models for each experiment with different hyperparameter combination, we are now going to perform a cross validation in the best models for each experiment, to have a more precise evaluation of the 

In [1]:
import tensorflow as tf

from numpy import genfromtxt
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection
from tensorflow import keras
from tensorflow.metrics import precision
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc
from sklearn.model_selection import KFold

features = genfromtxt('../datasets/final-data/features.csv', delimiter=',')
labels = genfromtxt('../datasets/final-data/labels.csv', delimiter=',')

## Selecting the best hyperparameter combination for each experiment

In [2]:
experiments = [dI for dI in os.listdir('../experiments') if os.path.isdir(os.path.join('../experiments',dI))]

best_models = []
for experiment in experiments:
    training_metrics = pd.read_csv('../experiments/' + experiment + '/training_metrics.csv')
    training_metrics = training_metrics.sort_values(['acc', 'precision', 'recall'], ascending=[False, False, False])
    best_model = training_metrics.iloc[0]
    best_model["experiment"] = experiment
    
    best_models.append(best_model)
    
pd.DataFrame(best_models)

/Users/gcarvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/gcarvs/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


,model_name,units,learning_rate,momentum,decay,activation_function,acc,loss,mae,mse,precision,recall,fs_score,experiment
535,model_536,35,0.0001,0.1000,0.0001,relu,0.556685,0.809226,0.5,0.314087,0.549858,0.605965,0.576550,pmean_title
344,model_345,75,0.0001,0.1000,0.1000,relu,0.548084,0.895111,0.5,0.329349,0.547812,0.530612,0.539075,pmean_body
261,model_262,75,0.0100,0.0100,0.0100,relu,0.578577,0.686320,0.5,0.262988,0.579032,0.563579,0.571201,mean_body
223,model_224,75,0.1000,0.0010,0.0001,relu,0.542611,0.736829,0.5,0.284417,0.548327,0.463108,0.502128,pmean_title_subtitle
471,model_472,35,0.0100,0.0001,0.0001,relu,0.558249,0.717674,0.5,0.275367,0.549315,0.629513,0.586686,mean_title
447,model_448,35,0.0100,0.0100,0.0001,relu,0.541830,0.704175,0.5,0.262183,0.550495,0.436421,0.486865,mean_title_subtitle


## Defining the helping functions

Here we are going to define five helper functions for the cross validation process:

1. create_model: creates a Keras model using the specified parameters
2. train_model: trains the models using the train dataset
3. test_model: tests the trained model using the test dataset
4. compute_c_validated_metrics: Computes the average and standard deviation of each metric
5. shuffle_dataset: shuffles the dataset

In [3]:
def create_model(model_name, units, activation_function, learning_rate, decay, momentum):
    linear_layer_size = 150
    if "pmean" in model_name:
        linear_layer_size = 450
    #Creating the network structure
    model = keras.Sequential()
    
    model.add(keras.layers.Input(shape=300,sparse=False))
    model.add(keras.layers.Dense(linear_layer_size))
    model.add(keras.layers.Dense(units, activation = activation_function))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(2, activation='softmax'))

    # Setting the optimizers parameters
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        decay=decay,
        momentum=momentum,
        nesterov=True
    )

    # Compiling the model
    model.compile(
        optimizer = optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['acc', 'mae', 'mse'])
    
    return model

def train_model(model, train_x, train_y):
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    
    history = model.fit(
        train_x, 
        train_y, 
        epochs = 40, 
        validation_split=0.3, 
        batch_size = 16,  
        verbose=1, 
        use_multiprocessing=True,
        callbacks=[early_stop]
    )
    
    return history

def test_model(model, test_x, test_y):
    loss, acc, mae, mse = model.evaluate(test_x, test_y, verbose=1)
    
    test_output_probabilities = model.predict(
        test_x,
        batch_size=16,
        verbose=1,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=True
    )
    
    predicted_y = np.argmax(test_output_probabilities, axis=1)
    
    average_precision = average_precision_score(test_y, predicted_y)
    precision_curve, recall_curve, _ = precision_recall_curve(test_y, predicted_y)
    
    fpr_rf, tpr_rf, _ = roc_curve(test_y, predicted_y)
    roc_auc_rf = auc(fpr_rf, tpr_rf)
    
    precision, recall, f_score, support = precision_recall_fscore_support(
        y_true = test_y, 
        y_pred = predicted_y, 
        average = 'binary'
    )
    
    return {
        "loss": loss,
        "acc": acc,
        "mae": mae,
        "mse": mse,
        "test_y": test_y,
        "predicted_y": predicted_y,
        "precision_curve": precision_curve,
        "recall_curve": recall_curve,
        "average_precision": average_precision,
        "fpr_rf": fpr_rf,
        "tpr_rf": tpr_rf,
        "roc_auc_rf": roc_auc_rf,
        "precision": precision,
        "recall": recall,
        "f_score": f_score
    }

def compute_c_validated_matrics(best_model, test_results):
    experiment_folder = "../cross_validation/" + best_model["experiment"]
    if not os.path.exists(experiment_folder):
        os.mkdir(experiment_folder)
    
    final_test_y = np.concatenate(test_results["test_y"], axis=0)
    np.savetxt(experiment_folder + "/true_y.csv", final_test_y, delimiter=",")
    
    final_predicted_y = np.concatenate(test_results["predicted_y"], axis=0)
    np.savetxt(experiment_folder + "/predicted_y.csv", final_predicted_y, delimiter=",")
    
    final_precision_curve = np.concatenate(test_results["precision_curve"], axis=0)
    np.savetxt(experiment_folder + "/precision_curve.csv", final_precision_curve, delimiter=",")
    
    final_recall_curve = np.concatenate(test_results["recall_curve"], axis=0)
    np.savetxt(experiment_folder + "/recall_curve.csv", final_recall_curve, delimiter=",")
    
    final_fpr_rf = np.concatenate(test_results["fpr_rf"], axis=0)
    np.savetxt(experiment_folder + "/fpr_rf.csv", final_fpr_rf, delimiter=",")
    
    final_tpr_rf = np.concatenate(test_results["tpr_rf"], axis=0)
    np.savetxt(experiment_folder + "/tpr_rf.csv", final_tpr_rf, delimiter=",")
    
    return {
        "experiment": best_model["experiment"],
        "model_name": best_model["model_name"],
        "units": best_model["units"], 
        "learning_rate": best_model["learning_rate"], 
        "momentum": best_model["momentum"],
        "decay": best_model["decay"],
        "activation_function": best_model["activation_function"],
        "average_acc": test_results["acc"].mean(),
        "acc_std_deviation": test_results["acc"].std(),
        "average_precision": test_results["precision"].mean(),
        "precision_std_deviation": test_results["precision"].std(),
        "average_recall": test_results["recall"].mean(),
        "recall_std_deviation": test_results["recall"].std(),
        "average_fs_score": test_results["f_score"].mean(),
        "fs_score_std_deviation": test_results["f_score"].std(),
        "average_precision_curve": test_results["average_precision"].mean(),
        "roc_auc_rf": test_results["roc_auc_rf"].mean()
    }

def shuffle_dataset(features, labels):
    shuffled_indexes = np.arange(len(features))
    np.random.shuffle(shuffled_indexes)
    shuffled_features = []
    shuffled_lables = []
    for index in shuffled_indexes:
        shuffled_features.append(features[index])
        shuffled_lables.append(labels[index])
    
    return np.array(shuffled_features), np.array(shuffled_lables)

## Cross validate the models

Each model will be cross validated using 10 folders of the data, and then we will compute the cross validated metrics.

In [4]:
cross_validated_models = []

for best_model in best_models:
    print(best_model["experiment"])
    cross_validation_results = []
    count = 1

    # Shuffle the dataset prior to initiating the validation
    features, labels = shuffle_dataset(features, labels)
    # We used k = 10 as it's the general number used in the literature
    for train_index,test_index in KFold(n_splits=10, random_state=None, shuffle=False).split(features):
        train_x, test_x = features[train_index], features[test_index]
        train_y, test_y = labels[train_index], labels[test_index]

        model = create_model(
            best_model['experiment'],
            best_model['units'], 
            best_model['activation_function'], 
            best_model['learning_rate'],
            best_model['decay'],
            best_model['momentum'])

        history = train_model(model, train_x, train_y)
        test_results = test_model(model, test_x, test_y)

        cross_validation_results.append(
            {
                "model_name": best_model["model_name"],
                "units": best_model["units"], 
                "learning_rate": best_model["learning_rate"], 
                "momentum": best_model["momentum"],
                "decay": best_model["decay"],
                "activation_function": best_model["activation_function"],
                "acc": test_results["acc"],
                "precision": test_results["precision"],
                "recall": test_results["recall"],
                "f_score": test_results["f_score"],
                "test_y": test_results["test_y"],
                "predicted_y": test_results["predicted_y"],
                "precision_curve": test_results["precision_curve"],
                "recall_curve": test_results["recall_curve"],
                "average_precision": test_results["average_precision"],
                "fpr_rf": test_results["fpr_rf"],
                "tpr_rf": test_results["tpr_rf"],
                "roc_auc_rf": test_results["roc_auc_rf"],
            }
        )
        print(count)
        count = count + 1

    cross_validation_df = pd.DataFrame(cross_validation_results)

    cross_validated_metrics = compute_c_validated_matrics(best_model, cross_validation_df)

    cross_validated_models.append(cross_validated_metrics)

W0526 20:21:35.712016 4614028736 deprecation.py:506] From /Users/gcarvs/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


pmean_title
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 0s 138us/sample
1
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 0s 144us/sample
2
pmean_body
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 0s 153us/sample
1
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 0s 147us/sample
2
mean_body
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 0s 176us/sample
1
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 0s 141us/sample
2
pmean_title_subtitle
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 1s 230us/sample
1
Train on 1712 samples, validate on 734 samples
2446/2446 [==============================] - 1s 237us/sample
2
mean_title
Train on 1712 samples, validate on 734 samples
2446/244

In [5]:
cross_validated_df = pd.DataFrame(cross_validated_models)
cross_validated_df = cross_validated_df[
    [
        "experiment", 
        "model_name",
        "units",
        "learning_rate", 
        "momentum",
        "decay",
        "activation_function",
        "average_acc",
        "acc_std_deviation",
        "average_precision",
        "precision_std_deviation",
        "average_recall",
        "recall_std_deviation",
        "average_fs_score",
        "fs_score_std_deviation",
        "average_precision_curve",
        "roc_auc_rf",
    ]]
cross_validated_df

,experiment,model_name,units,learning_rate,momentum,decay,activation_function,average_acc,acc_std_deviation,average_precision,precision_std_deviation,average_recall,recall_std_deviation,average_fs_score,fs_score_std_deviation,average_precision_curve,roc_auc_rf
0,pmean_title,model_536,35,0.0001,0.1000,0.0001,relu,0.509403,0.000578,0.509037,0.001355,0.634015,0.239993,0.554616,0.098152,0.506056,0.507506
1,pmean_body,model_345,75,0.0001,0.1000,0.1000,relu,0.485282,0.002313,0.486642,0.008279,0.492128,0.132138,0.485461,0.070077,0.494943,0.484910
2,mean_body,model_262,75,0.0100,0.0100,0.0100,relu,0.511652,0.019369,0.515596,0.026376,0.652726,0.211999,0.565947,0.067726,0.508931,0.512687
3,pmean_title_subtitle,model_224,75,0.1000,0.0010,0.0001,relu,0.506337,0.010696,0.510283,0.008516,0.376728,0.125269,0.428108,0.086794,0.505935,0.507157
4,mean_title,model_472,35,0.0100,0.0001,0.0001,relu,0.507155,0.002024,0.535063,0.027088,0.221213,0.206197,0.280832,0.209412,0.506208,0.507428
5,mean_title_subtitle,model_448,35,0.0100,0.0100,0.0001,relu,0.502044,0.001735,0.502486,0.001950,0.884895,0.053198,0.640685,0.012382,0.502445,0.500393


In [6]:
cross_validated_df.to_csv("../cross_validation/cross_validation_metrics.csv", index= False, encoding='utf-8')